In [1]:
import torch
torch.use_deterministic_algorithms(True)

In [2]:
B=4
T=8
C=2
torch.manual_seed(1337)
x=torch.randn((B,T,C),dtype=torch.float64) # B,T,C
x.shape

torch.Size([4, 8, 2])

In [3]:
#xbow[b,t,c] = avg x[b,x<=t,c] We are tryung to do this effecciently
xbow=torch.zeros(4,8,2,dtype=torch.float64)
for b in range(B):
    for t in range(T):
        tmp=x[b,:t+1] ## t*C
        xbow[b,t]=tmp.mean(dim=0)

In [4]:
xtemp=torch.zeros(4,8,2,dtype=torch.float64)
for t in range(T):
    tmp=x[:,:t+1,:]
    xtemp[:,t,:]=tmp.mean(dim=1)

In [5]:
torch.allclose(xtemp,xbow)

True

In [ ]:
## Averages via matrix multiplication assuming there is no batch dimension
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3,dtype=torch.float64))
# a = a/a.sum(dim=1,keepdim=True)
b = torch.randint(0,10,(3,2),dtype=torch.float64)

c= a@b
print(f"a= {a}")
print(f"b= {b}")
print(f"b= {c}")


# T * T and B * T* C matrix were multiplied the multiplication was broadcasted over the batch dimension

In [6]:
## gettting equal to xbow using matrix multiply aggregation
wei=torch.tril(torch.ones(T,T,dtype=torch.float64)) #lower triangular all ones matrix
wei=wei/wei.sum(1,keepdim=True)
print("wei shape",wei.shape)
print("x shape",x.shape)
xbow2=wei@x # wei is T*T x is B*T*C pytorch will interpret B as batch dimension and brodcast over it


wei shape torch.Size([8, 8])
x shape torch.Size([4, 8, 2])


In [7]:
torch.allclose(xbow,xbow2)

True

# Via softmax this will end up being used

In [8]:
tril=torch.tril(torch.ones(T,T,dtype=torch.float64))
wei2=torch.zeros(T,T,dtype=torch.float64).masked_fill(tril==0,float('-inf'))
wei2=wei2.softmax(dim=1)
torch.allclose(wei,wei2)

True